## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])

# Display the first few rows of the modified dataframe
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
# unique_value_counts = application_df.nunique()
for x in application_df.columns:
    print(x, len(application_df[x].unique()))

APPLICATION_TYPE 17
AFFILIATION 6
CLASSIFICATION 71
USE_CASE 5
ORGANIZATION 4
STATUS 2
INCOME_AMT 9
SPECIAL_CONSIDERATIONS 2
ASK_AMT 8747
IS_SUCCESSFUL 2


In [4]:
# Look at APPLICATION_TYPE value counts for binning
counts = application_df['APPLICATION_TYPE'].value_counts()
counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

# Calculate the counts for each application type
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()

# Define a cutoff value
xcutoff_value = 500  

# Create a list of application types to be replaced with "Other"
application_types_to_replace = list(counts[counts<500].index)
#application_types_to_replace = application_type_counts[application_type_counts < xcutoff_value].index.tolist()

# Display the application types to be replaced
print(application_types_to_replace)

# Replace in dataframe using a dictionary
replacement_dict_app = {app: 'Other' for app in application_types_to_replace}
application_df['APPLICATION_TYPE'].replace(replacement_dict_app, inplace=True)

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']


APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts_gt1 = classification_counts[classification_counts > 1]
classification_counts_gt1

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
# Define a cutoff value
cutoff_value = 100  

# Create a list of classifications to be replaced with "Other"
classifications_to_replace = classification_counts_gt1[classification_counts_gt1 < cutoff_value].index.tolist()

# Display the classifications to be replaced
print(classifications_to_replace)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

['C2800', 'C7100', 'C1300', 'C1280', 'C1230', 'C1400', 'C7200', 'C2300', 'C1240', 'C8000', 'C7120', 'C1500', 'C1800', 'C6000', 'C1250', 'C8200', 'C1238', 'C1278', 'C1235', 'C1237', 'C7210', 'C2400', 'C1720', 'C4100', 'C1257', 'C1600', 'C1260', 'C2710', 'C0', 'C3200', 'C1234', 'C1246', 'C1267', 'C1256']


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      643
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C1370        1
C1236        1
C2190        1
C4200        1
C5200        1
C1900        1
C2600        1
C1248        1
C6100        1
C1820        1
C1580        1
C2570        1
C1732        1
C3700        1
C2500        1
C1570        1
C1283        1
C2380        1
C1728        1
C2170        1
C4120        1
C8210        1
C4500        1
C1245        1
C2561        1
C2150        1
Name: count, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_encoded = pd.get_dummies(application_df, columns=['APPLICATION_TYPE', 'AFFILIATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS', 'CLASSIFICATION'])

# Display the first few rows of the encoded DataFrame
application_df_encoded.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,CLASSIFICATION_C4000,CLASSIFICATION_C4120,CLASSIFICATION_C4200,CLASSIFICATION_C4500,CLASSIFICATION_C5000,CLASSIFICATION_C5200,CLASSIFICATION_C6100,CLASSIFICATION_C7000,CLASSIFICATION_C8210,CLASSIFICATION_Other
0,1,5000,1,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1,108590,1,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,1,5000,0,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
3,1,6692,1,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,1,142590,1,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [10]:
# Split our preprocessed data into our features and target arrays
# Define X (features) and y (target)
X = application_df_encoded.drop(columns=['IS_SUCCESSFUL'])  
y = application_df_encoded['IS_SUCCESSFUL']  
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Display the shapes of the training and testing datasets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


X_train shape: (25724, 75)
X_test shape: (8575, 75)
y_train shape: (25724,)
y_test shape: (8575,)


In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len( X_train_scaled[0])
#input_features = X.shape[1]
nn = tf.keras.models.Sequential()
nn = Sequential()

# First hidden layer
hidden_layer_1_nodes = 7
nn.add(Dense(units=hidden_layer_1_nodes, input_dim=input_features, activation='relu'))

# Second hidden layer
hidden_layer_2_nodes = 14
nn.add(Dense(units=hidden_layer_2_nodes, activation='relu'))

# Third hidden layer
#hidden_layer_3_nodes = 21
#nn.add(Dense(units=hidden_layer_3_nodes, activation='relu'))

# Output layer
nn.add(Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 7)                 532       
                                                                 
 dense_1 (Dense)             (None, 14)                112       
                                                                 
 dense_2 (Dense)             (None, 1)                 15        
                                                                 
Total params: 659 (2.57 KB)
Trainable params: 659 (2.57 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
# Error checking to display the data types of features and labels
print("Data type of X_train:", X_train.dtypes.unique())
print("Data type of y_train:", y_train.dtypes)

Data type of X_train: [dtype('int64') dtype('bool')]
Data type of y_train: int64


In [15]:
# Convert boolean values in features to integers and overwrite X_train
X_train = X_train.astype('int64')
# Convert labels to NumPy array with appropriate data type
y_train_array = np.asarray(y_train).astype('float32')

In [16]:
# Train the model
nn.fit(X_train, y_train_array, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 13187.3496 - accuracy: 0.5013
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 11810.5850 - accuracy: 0.4891
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 6949.8462 - accuracy: 0.4881
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 5169.0400 - accuracy: 0.5239
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 8409.0166 - accuracy: 0.4972
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 3544.0730 - accuracy: 0.5170
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 7189.0205 - accuracy: 0.5088
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 3162.5781 - accuracy: 0.5251
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 456.7788 - accuracy: 0.5219
Epoch 10/100
804/804 [==============================] - 2s 2ms/

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.7372 - accuracy: 0.5234 - 613ms/epoch - 2ms/step
Loss: 0.7371546626091003, Accuracy: 0.5233819484710693


In [20]:
import os
import tensorflow as tf

# Specify the desired directory path
save_directory = 'C:/Users/Richards/Desktop/Data Analyst/Projects/deep-learning-challenge/deep-learning-challenge'

# Combine the directory path and the file name
save_path = os.path.join(save_directory, 'AlphabetSoupCharity.h5')

# Save the model in the specified location
nn.save(save_path)

# Display a message indicating that the model has been saved
print(f"Model has been exported to {save_path}")


Model has been exported to C:/Users/Richards/Desktop/Data Analyst/Projects/deep-learning-challenge/deep-learning-challenge\AlphabetSoupCharity.h5


c:\Users\Richards\.conda\envs\dev\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
